In [ ]:
# Instalar dependencias
!apt-get install -y unixodbc-dev
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/20.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql17

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import sessionmaker
import pymssql
import gspread
from google.auth import default
from io import BytesIO
from googleapiclient.discovery import build

def main(request):
    try:
        # Inicializar Google Drive y autenticación
        creds, _ = default()
        gc = gspread.authorize(creds)

        # Abrir el archivo CSV desde Google Drive (reemplazar con tu ID de archivo)
        file_id = '1kF3hIEwPFojn2Dk_dMd1sHg_JWXAZ7Z2'  # Reemplazar con tu ID real

        # Usar Google Drive API para obtener el archivo
        drive_service = build('drive', 'v3', credentials=creds)
        request = drive_service.files().get_media(fileId=file_id)
        file_data = request.execute()

        # Leer contenido CSV en un DataFrame
        df = pd.read_csv(BytesIO(file_data))

        # Conexión a la base de datos SQL Server
        user = 'sqlserver'
        password = 'Contraseña1'
        server = '34.135.222.123'
        database = 'nbaplayers'

        # Crear la cadena de conexión
        connection_string = f"mssql+pymssql://{user}:{password}@{server}/{database}"
        engine = create_engine(connection_string)

        # Crear la sesión
        Session = sessionmaker(bind=engine)

        # Obtener las fechas existentes en la base de datos
        with Session() as session:
            fechas_existentes_query = "SELECT DISTINCT ingestion_date FROM MH2024season"
            fechas_existentes = pd.read_sql(fechas_existentes_query, session.bind)

        # Filtrar los nuevos registros basados en las fechas existentes
        nuevos_registros = df[~df['ingestion_date'].isin(fechas_existentes['ingestion_date'])]

        def insertar_nuevos_registros(df, table_name, engine):
            columnas_float = ['PTS_Regular', 'PTS_Playoffs', 'REB_Regular', 'REB_Playoffs',
                            'AST_Regular', 'AST_Playoffs', 'TO_Regular', 'TO_Playoffs',
                            '_3PM_Regular', '_3PM_Playoffs', 'FT_Regular', 'FT_Playoffs']

            for col in columnas_float:
                df[col] = pd.to_numeric(df[col], errors='coerce')

            try:
                with engine.begin() as conn:
                    df.to_sql(table_name, conn, if_exists='append', index=False)
                    return f"Se insertaron {len(df)} nuevos registros."
            except IntegrityError as e:
                return f"Error de integridad: {e}"
            except Exception as e:
                return f"Error al insertar registros: {e}"

        if not nuevos_registros.empty:
            resultado = insertar_nuevos_registros(nuevos_registros, 'MH2024season', engine)
            return f"{resultado}"
        else:
            return "No hay nuevos registros para insertar."
    except Exception as e:
        print(f"Error al ejecutar la función: {e}")
        return "Error interno del servidor", 500


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 msodbcsql17 : Depends: odbcinst but it is not going to be installed
 unixodbc : Depends: odbcinst1debian2 (>= 2.3.11-1) but it is not going to be installed
            Depends: libodbc1 (>= 2.3.11-1) but it is not going to be installed
 unixodbc-dev : Depends: odbcinst1debian2 (= 2.3.11-1) but it is not going to be installed
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0 